In [5]:
import cv2
import face_alignment
import numpy as np
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import speech_recognition as sr
import librosa
from scipy.spatial import distance
from collections import deque


# ==================================
# Video Analyzer
# ==================================
class VideoAnalyzer:
    def _init_(self):
        self.total_blinks = 0
        self.prev_ear = 0
        self.blink_thresh = 0.2275
        self.prev_head_dir = 0
        self.head_turn_thresh = 0.25
        self.sus_head_movements = 0
        self.gaze_warnings = 0
        self.gaze_history = deque(maxlen=10)
        self.center_x = 0.5
        self.calibrated = False
        self.start_time = time.time()

    def eye_aspect_ratio(self, eye):
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        return (A + B) / (2.0 * C)

    def detect_pupil(self, eye_img):
        if eye_img.size == 0:
            return None
        gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            return None
        c = max(contours, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
        return (x + w // 2, y + h // 2)

    def crop_eye(self, frame, eye_points):
        x_min, x_max = int(np.min(eye_points[:, 0])), int(np.max(eye_points[:, 0]))
        y_min, y_max = int(np.min(eye_points[:, 1])), int(np.max(eye_points[:, 1]))
        return frame[y_min:y_max, x_min:x_max]

    def get_head_direction(self, landmarks):
        nose, chin = landmarks[30], landmarks[8]
        return np.arctan2(chin[1] - nose[1], chin[0] - nose[0])

    def calibrate_center(self, frame, landmarks):
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]
        eyes = [left_eye, right_eye]
        x_vals = []
        for eye_points in eyes:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil:
                norm_x = pupil[0] / (eye_img.shape[1] + 1e-6)
                x_vals.append(norm_x)
        if x_vals:
            self.center_x = np.mean(x_vals)
            self.calibrated = True

    def process_landmarks(self, frame, landmarks):
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]

        # Blink detection
        ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
        if ear < self.blink_thresh and self.prev_ear >= self.blink_thresh:
            self.total_blinks += 1
        self.prev_ear = ear

        # Head movement
        head_dir = self.get_head_direction(landmarks)
        if abs(head_dir - self.prev_head_dir) > self.head_turn_thresh:
            self.sus_head_movements += 1
        self.prev_head_dir = head_dir

        # Eye gaze
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil:
                norm_x = pupil[0] / (eye_img.shape[1] + 1e-6)
                self.gaze_history.append(norm_x)

        if len(self.gaze_history) > 0:
            avg_x = np.mean(self.gaze_history)
            if avg_x < self.center_x - 0.1 or avg_x > self.center_x + 0.06:
                self.gaze_warnings += 1

    def final_report(self):
        prob_ai = (
            (self.gaze_warnings * 0.4) +
            (self.sus_head_movements * 0.3) +
            (self.total_blinks * 0.3)
        )
        prob_ai = min(prob_ai / 100.0, 1.0)
        return round(prob_ai, 2)


# ==================================
# Speech Analyzer
# ==================================
class SpeechAnalyzer:
    def _init_(self):
        self.score = 0.0

    def analyze_audio(self, duration=5):
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("🎙 Speak now...")
            audio = r.record(source, duration=duration)
        try:
            text = r.recognize_google(audio)
            print("Transcript:", text)
            y, sr_rate = librosa.load(sr.AudioData.get_wav_data(audio), sr=None)
            pitch = librosa.yin(y, fmin=50, fmax=300)
            pitch_std = np.std(pitch)
            if pitch_std < 5: self.score += 0.4
            if len(text.split()) / duration < 1.5: self.score += 0.3
            if len(text) == 0: self.score += 0.3
        except:
            self.score = 0.5
        return round(min(self.score, 1.0), 2)


# ==================================
# Text Analyzer
# ==================================
class TextAnalyzer:
    def _init_(self, model_path="./results_improved/final_model"):
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.detector = pipeline("text-classification", model=model, tokenizer=tokenizer)

    def analyze(self, text):
        result = self.detector(text)[0]
        return round(result['score'], 2)


# ==================================
# Fusion Layer
# ==================================
def fusion_layer(text_score, speech_score, video_score):
    overall_score = 0.4*text_score + 0.4*speech_score + 0.2*video_score
    if overall_score < 0.45:
        label, color = "Likely Human", "Green"
    elif overall_score < 0.65:
        label, color = "Uncertain", "Yellow"
    else:
        label, color = "Likely AI / Suspicious", "Red"
    return {
        "text_score": text_score,
        "speech_score": speech_score,
        "video_score": video_score,
        "overall_score": round(overall_score, 2),
        "verdict": label,
        "color": color
    }


# ==================================
# Main
# ==================================
def main():
    # Init analyzers
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False)
    video_analyzer = VideoAnalyzer()
    speech_analyzer = SpeechAnalyzer()
    text_analyzer = TextAnalyzer()

    # --- Calibration (video) ---
    cap = cv2.VideoCapture(0)
    print("Calibrating gaze... look at CENTER")
    start_calib = time.time()
    while time.time() - start_calib < 3:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.calibrate_center(frame, preds[0])
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    print("Calibration done ✅")

    # --- Video collection ---
    start_video = time.time()
    while time.time() - start_video < 5:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.process_landmarks(frame, preds[0])
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    cap.release()
    cv2.destroyAllWindows()
    video_score = video_analyzer.final_report()

    # --- Speech ---
    speech_score = speech_analyzer.analyze_audio(duration=5)

    # --- Text ---
    try:
        with open("transcription.txt") as f:
            text = f.read().strip()
    except:
        text = "Sample placeholder text"
    text_score = text_analyzer.analyze(text)

    # --- Fusion ---
    report = fusion_layer(text_score, speech_score, video_score)
    print("\n🔥 FINAL REPORT 🔥")
    for k, v in report.items():
        print(f"{k}: {v}")


if _name_ == "_main_":
    main()

NameError: name '_name_' is not defined

In [ ]:
import cv2
import face_alignment
import numpy as np
import time
import wave
import pyaudio
import librosa
import speech_recognition as sr
from scipy.spatial import distance
from collections import deque
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


# ==================================
# Video Analyzer
# ==================================
class VideoAnalyzer:
    def __init__(self):
        self.total_blinks = 0
        self.prev_ear = 0
        self.blink_thresh = 0.2275
        self.prev_head_dir = 0
        self.head_turn_thresh = 0.25
        self.sus_head_movements = 0
        self.gaze_warnings = 0
        self.gaze_history = deque(maxlen=10)
        self.center_x = 0.5
        self.calibrated = False

    def eye_aspect_ratio(self, eye):
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        return (A + B) / (2.0 * C)

    def detect_pupil(self, eye_img):
        if eye_img.size == 0:
            return None
        gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            return None
        c = max(contours, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
        return (x + w // 2, y + h // 2)

    def crop_eye(self, frame, eye_points):
        x_min, x_max = int(np.min(eye_points[:, 0])), int(np.max(eye_points[:, 0]))
        y_min, y_max = int(np.min(eye_points[:, 1])), int(np.max(eye_points[:, 1]))
        return frame[y_min:y_max, x_min:x_max]

    def get_head_direction(self, landmarks):
        nose, chin = landmarks[30], landmarks[8]
        return np.arctan2(chin[1] - nose[1], chin[0] - nose[0])

    def calibrate_center(self, frame, landmarks):
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]
        eyes = [left_eye, right_eye]
        x_vals = []
        for eye_points in eyes:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil:
                norm_x = pupil[0] / (eye_img.shape[1] + 1e-6)
                x_vals.append(norm_x)
        if x_vals:
            self.center_x = np.mean(x_vals)
            self.calibrated = True

    def process_landmarks(self, frame, landmarks):
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]

        # Blink detection
        ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
        if ear < self.blink_thresh and self.prev_ear >= self.blink_thresh:
            self.total_blinks += 1
        self.prev_ear = ear

        # Head movement
        head_dir = self.get_head_direction(landmarks)
        if abs(head_dir - self.prev_head_dir) > self.head_turn_thresh:
            self.sus_head_movements += 1
        self.prev_head_dir = head_dir

        # Eye gaze
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil:
                norm_x = pupil[0] / (eye_img.shape[1] + 1e-6)
                self.gaze_history.append(norm_x)

        if len(self.gaze_history) > 0:
            avg_x = np.mean(self.gaze_history)
            if avg_x < self.center_x - 0.1 or avg_x > self.center_x + 0.06:
                self.gaze_warnings += 1

    def final_report(self):
        prob_ai = (
            (self.gaze_warnings * 0.4) +
            (self.sus_head_movements * 0.3) +
            (self.total_blinks * 0.3)
        )
        prob_ai = min(prob_ai / 100.0, 1.0)
        return round(prob_ai, 2)


# ==================================
# Audio Recorder
# ==================================
def record_audio(filename="output_audio.wav"):
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("🎥 Interview recording... Press 'q' to stop video window.")
    frames = []

    while True:
        data = stream.read(CHUNK)
        frames.append(data)

        # Stop when video window is closed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print("Audio recording stopped.")
    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()


# ==================================
# Speech Analyzer
# ==================================
class SpeechAnalyzer:
    def __init__(self):
        self.score = 0.0

    def analyze_audio(self, filename="output_audio.wav"):
        try:
            # Transcribe
            r = sr.Recognizer()
            with sr.AudioFile(filename) as source:
                audio = r.record(source)
            text = r.recognize_google(audio)
            print("Transcript:", text)

            # Pitch analysis
            y, sr_rate = librosa.load(filename, sr=None)
            pitch = librosa.yin(y, fmin=50, fmax=300)
            pitch_std = np.std(pitch)
            if pitch_std < 5: self.score += 0.4
            if len(text.split()) / (len(y) / sr_rate) < 1.5: self.score += 0.3
            if len(text) == 0: self.score += 0.3

            return round(min(self.score, 1.0), 2), text
        except:
            return 0.5, ""


# ==================================
# Text Analyzer
# ==================================
class TextAnalyzer:
    def __init__(self, model_path="./final_model"):
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.detector = pipeline("text-classification", model=model, tokenizer=tokenizer)

    def analyze(self, text):
        if not text.strip():
            return 0.5
        result = self.detector(text)[0]
        return round(result['score'], 2)


# ==================================
# Fusion Layer
# ==================================
def fusion_layer(text_score, speech_score, video_score):
    overall_score = 0.4*text_score + 0.4*speech_score + 0.2*video_score
    if overall_score < 0.45:
        label = "Likely Human"
    elif overall_score < 0.65:
        label = "Uncertain"
    else:
        label = "Likely AI / Suspicious"
    return {
        "text_score": text_score,
        "speech_score": speech_score,
        "video_score": video_score,
        "overall_score": round(overall_score, 2),
        "verdict": label
    }


# ==================================
# Main Pipeline
# ==================================
def main():
    # Init
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False)
    video_analyzer = VideoAnalyzer()

    # Open video & audio
    cap = cv2.VideoCapture(0)

    # Calibration
    print("Calibrating gaze... look at CENTER")
    while not video_analyzer.calibrated:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.calibrate_center(frame, preds[0])
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    print("Calibration done ✅")

    # Start recording audio in parallel
    record_audio("output_audio.wav")

    # Process video until stopped
    while True:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.process_landmarks(frame, preds[0])
        cv2.imshow("Interview", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()
    video_score = video_analyzer.final_report()

    # Speech
    speech_analyzer = SpeechAnalyzer()
    speech_score, transcript = speech_analyzer.analyze_audio("output_audio.wav")

    # Text
    text_analyzer = TextAnalyzer()
    text_score = text_analyzer.analyze(transcript)

    # Fusion
    report = fusion_layer(text_score, speech_score, video_score)
    print("\n🔥 FINAL REPORT 🔥")
    for k, v in report.items():
        print(f"{k}: {v}")


if __name__ == "__main__":
    main()


🎥 Recording interview...


Device set to use cuda:0



🔥 FINAL REPORT 🔥
text_score: 1.0
speech_score: 0.3
video_score: 0.0
overall_score: 0.52
verdict: Uncertain


In [ ]:
import cv2
import torch
import face_alignment
import numpy as np
from scipy.spatial import distance
from collections import deque
import time

# -------------------------------
# Helper Functions
# -------------------------------

def eye_aspect_ratio(eye):
    """Compute EAR for blink detection."""
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def crop_eye(frame, eye_points):
    """Crop the eye region using bounding rect."""
    x_min = int(np.min(eye_points[:, 0]))
    x_max = int(np.max(eye_points[:, 0]))
    y_min = int(np.min(eye_points[:, 1]))
    y_max = int(np.max(eye_points[:, 1]))
    return frame[y_min:y_max, x_min:x_max], (x_min, y_min, x_max, y_max)

def detect_pupil(eye_img):
    """Detect pupil center using threshold + contours."""
    if eye_img.size == 0:
        return None
    gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    # Largest contour = pupil
    c = max(contours, key=cv2.contourArea)
    (x, y, w, h) = cv2.boundingRect(c)
    cx, cy = x + w // 2, y + h // 2
    return (cx, cy)

def get_head_direction(landmarks):
    """Rough head roll detection (nose-chin slope)."""
    nose = landmarks[30]
    chin = landmarks[8]
    slope = np.arctan2(chin[1] - nose[1], chin[0] - nose[0])
    return slope

# -------------------------------
# Analyzer Class
# -------------------------------

class VideoAnalyzer:
    def _init_(self):
        self.total_blinks = 0
        self.prev_ear = 0
        self.blink_thresh = 0.2275

        self.prev_head_dir = 0
        self.head_turn_thresh = 0.25
        self.sus_head_movements = 0

        self.gaze_history = deque(maxlen=10)
        self.gaze_counts = {"LEFT": 0, "RIGHT": 0, "CENTER": 0}
        self.gaze_warnings = 0

        self.start_time = time.time()

        # Calibration
        self.center_x = 0.5  # default normalized center
        self.calibrated = False

    def calibrate_center(self, frame, landmarks):
        """Calibrate gaze by asking user to look at the center."""
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        eyes = [left_eye, right_eye]
        x_vals = []

        for eye_points in eyes:
            eye_img, _ = crop_eye(frame, eye_points)
            pupil = detect_pupil(eye_img)
            if pupil is not None:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                x_vals.append(norm_x)

        if x_vals:
            self.center_x = np.mean(x_vals)
            self.calibrated = True

    def process_landmarks(self, frame, landmarks):
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        # Blink detection
        ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
        if ear < self.blink_thresh and self.prev_ear >= self.blink_thresh:
            self.total_blinks += 1
        self.prev_ear = ear

        # Head movement
        head_dir = get_head_direction(landmarks)
        if abs(head_dir - self.prev_head_dir) > self.head_turn_thresh:
            self.sus_head_movements += 1
        self.prev_head_dir = head_dir

        # Eye gaze detection
        gaze_direction = "CENTER"
        for eye_points in [left_eye, right_eye]:
            eye_img, _ = crop_eye(frame, eye_points)
            pupil = detect_pupil(eye_img)
            if pupil is not None:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                self.gaze_history.append(norm_x)

        if len(self.gaze_history) > 0:
            avg_x = np.mean(self.gaze_history)

            if avg_x < self.center_x - 0.1:
                gaze_direction = "RIGHT"
            elif avg_x > self.center_x + 0.06:
                gaze_direction = "LEFT"
            else:
                gaze_direction = "CENTER"

            self.gaze_counts[gaze_direction] += 1

        # Warning only for left/right
        warning = None
        if gaze_direction in ["LEFT", "RIGHT"]:
            warning = f"⚠ GAZE WARNING: {gaze_direction}"
            self.gaze_warnings += 1

        return warning

    def final_report(self):
        elapsed = round(time.time() - self.start_time, 2)
        # Simple AI-use probability scoring
        prob_ai = (
            (self.gaze_warnings * 0.4) +
            (self.sus_head_movements * 0.3) +
            (self.total_blinks * 0.3)
        )
        prob_ai = min(prob_ai / 100.0, 1.0)  # Normalize 0–1

        return {
            "Total Time (s)": elapsed,
            "Total Blinks": self.total_blinks,
            "Suspected Head Movements": self.sus_head_movements,
            "AI-Use Probability": round(prob_ai, 2)
        }

# -------------------------------
# Main Pipeline
# -------------------------------

def main():
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False)
    analyzer = VideoAnalyzer()

    cap = cv2.VideoCapture(0)
    print("✅ Video monitoring started. Look at the CENTER for calibration...")

    # Calibration phase
    start_calib = time.time()
    while time.time() - start_calib < 3:  # 3 seconds calibration
        ret, frame = cap.read()
        if not ret:
            break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            analyzer.calibrate_center(frame, preds[0])
        cv2.putText(frame, "Look at CENTER for calibration", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.imshow("Video Analysis", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    print("✅ Calibration complete.")

    # Main monitoring
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        preds = fa.get_landmarks(frame)
        if preds is not None:
            landmarks = preds[0]
            warning = analyzer.process_landmarks(frame, landmarks)

            if warning:
                cv2.putText(frame, warning, (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Video Analysis", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Final report
    final_report = analyzer.final_report()
    print("\n✅ Final Report:")
    for key, val in final_report.items():
        print(f"{key}: {val}")

if _name_ == "_main_":
    main()

In [ ]:
import cv2
import face_alignment
import numpy as np
import threading
import wave
import pyaudio
import time
import librosa
import speech_recognition as sr
from scipy.spatial import distance
from collections import deque
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# ==================================
# Video Analyzer (UPDATED based on your new code)
# ==================================
class VideoAnalyzer:
    def __init__(self):
        # General
        self.start_time = time.time()

        # Blink Detection
        self.total_blinks = 0
        self.prev_ear = 0
        self.blink_thresh = 0.2275

        # Head Movement
        self.prev_head_dir = 0
        self.head_turn_thresh = 0.25
        self.sus_head_movements = 0

        # Gaze Tracking
        self.gaze_history = deque(maxlen=10)
        self.gaze_counts = {"LEFT": 0, "RIGHT": 0, "CENTER": 0}
        self.gaze_warnings = 0

        # Calibration
        self.center_x = 0.5  # default normalized center
        self.calibrated = False

    def eye_aspect_ratio(self, eye):
        """Compute EAR for blink detection."""
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        return (A + B) / (2.0 * C)

    def crop_eye(self, frame, eye_points):
        """Crop the eye region using bounding rect."""
        x_min = int(np.min(eye_points[:, 0]))
        x_max = int(np.max(eye_points[:, 0]))
        y_min = int(np.min(eye_points[:, 1]))
        y_max = int(np.max(eye_points[:, 1]))
        return frame[y_min:y_max, x_min:x_max]

    def detect_pupil(self, eye_img):
        """Detect pupil center using threshold + contours."""
        if eye_img.size == 0:
            return None
        gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            return None

        # Largest contour = pupil
        c = max(contours, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
        return (x + w // 2, y + h // 2)

    def get_head_direction(self, landmarks):
        """Rough head roll detection (nose-chin slope)."""
        nose, chin = landmarks[30], landmarks[8]
        return np.arctan2(chin[1] - nose[1], chin[0] - nose[0])

    def calibrate_center(self, frame, landmarks):
        """Calibrate gaze by asking user to look at the center."""
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]
        x_vals = []
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil is not None and eye_img.shape[1] > 0:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                x_vals.append(norm_x)
        if x_vals:
            self.center_x = np.mean(x_vals)
            self.calibrated = True

    def process_landmarks(self, frame, landmarks):
        """Process landmarks to detect blinks, head movement, and eye gaze."""
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]

        # Blink detection
        ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
        if ear < self.blink_thresh and self.prev_ear >= self.blink_thresh:
            self.total_blinks += 1
        self.prev_ear = ear

        # Head movement
        head_dir = self.get_head_direction(landmarks)
        if self.prev_head_dir != 0 and abs(head_dir - self.prev_head_dir) > self.head_turn_thresh:
            self.sus_head_movements += 1
        self.prev_head_dir = head_dir

        # Eye gaze detection
        gaze_direction = "CENTER"
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil is not None and eye_img.shape[1] > 0:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                self.gaze_history.append(norm_x)

        if len(self.gaze_history) > 0:
            avg_x = np.mean(self.gaze_history)
            if avg_x < self.center_x - 0.1:
                gaze_direction = "RIGHT"  # User looks right, pupils move to left of eye frame
            elif avg_x > self.center_x + 0.06:
                gaze_direction = "LEFT" # User looks left, pupils move to right of eye frame
            else:
                gaze_direction = "CENTER"
            self.gaze_counts[gaze_direction] += 1

        # Return a warning for immediate feedback, only for off-center gaze
        if gaze_direction in ["LEFT", "RIGHT"]:
            self.gaze_warnings += 1
            return f"⚠ GAZE WARNING: {gaze_direction}"
        return None

    def final_report(self):
        """Generate a final dictionary with all video analysis stats."""
        elapsed = round(time.time() - self.start_time, 2)
        # Simple AI-use probability scoring
        prob_ai = (
            (self.gaze_warnings * 0.4) +
            (self.sus_head_movements * 0.3) +
            (self.total_blinks * 0.3)
        )
        prob_ai = min(prob_ai / 100.0, 1.0)  # Normalize 0–1

        return {
            "Total Time (s)": elapsed,
            "Total Blinks": self.total_blinks,
            "Suspected Head Movements": self.sus_head_movements,
            "AI-Use Probability": round(prob_ai, 2)
        }

# ==================================
# Audio Recorder (No changes needed)
# ==================================
def record_audio(stop_event, filename="output_audio.wav"):
    FORMAT, CHANNELS, RATE, CHUNK = pyaudio.paInt16, 1, 16000, 1024
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("🎤 Audio recording started...")
    frames = []
    while not stop_event.is_set():
        data = stream.read(CHUNK)
        frames.append(data)
    print("🎤 Audio recording stopped. Saving file...")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    print(f"🎤 Audio saved to {filename}")


# ==================================
# Speech Analyzer (No changes needed)
# ==================================
class SpeechAnalyzer:
    def __init__(self):
        self.score = 0.0

    def analyze_audio(self, filename="output_audio.wav"):
        try:
            r = sr.Recognizer()
            with sr.AudioFile(filename) as source:
                audio = r.record(source)
            text = r.recognize_google(audio)
            print("📝 Transcript:", text)
            y, sr_rate = librosa.load(filename, sr=None)
            pitch = librosa.yin(y, fmin=50, fmax=300)
            pitch_std = np.std(pitch[~np.isnan(pitch)])
            if pitch_std < 5: self.score += 0.4
            if len(y) > 0 and len(text.split()) / (len(y) / sr_rate) < 1.5: self.score += 0.3
            if not text: self.score += 0.3
            return round(min(self.score, 1.0), 2), text
        except Exception as e:
            print(f"Speech analysis failed: {e}. Returning default values.")
            return 0.5, ""


# ==================================
# Text Analyzer (No changes needed)
# ==================================
class TextAnalyzer:
    def __init__(self, model_path="distilbert-base-uncased-finetuned-sst-2-english"):
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.detector = pipeline("text-classification", model=model, tokenizer=tokenizer)

    def analyze(self, text):
        if not text.strip():
            return 0.5
        result = self.detector(text)[0]
        score = result['score'] if result['label'] == 'POSITIVE' else 1 - result['score']
        return round(score, 2)


# ==================================
# Fusion Layer (No changes needed)
# ==================================
def fusion_layer(text_score, speech_score, video_score):
    overall_score = 0.4 * text_score + 0.4 * speech_score + 0.2 * video_score
    label = "Uncertain"
    if overall_score < 0.45:
        label = "Likely Human"
    elif overall_score > 0.65:
        label = "Likely AI / Suspicious"
    return {
        "text_score": text_score,
        "speech_score": speech_score,
        "video_score": video_score,
        "overall_score": round(overall_score, 2),
        "verdict": label
    }


# ==================================
# Main Pipeline (UPDATED)
# ==================================
def main():
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu')
    video_analyzer = VideoAnalyzer()
    cap = cv2.VideoCapture(0)

    # Calibration phase
    print("Calibrating gaze... look at the CENTER for 3 seconds.")
    start_calib = time.time()
    while time.time() - start_calib < 3:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.calibrate_center(frame, preds[0])
        cv2.putText(frame, "Look at CENTER for calibration", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    
    cv2.destroyWindow("Calibration")
    if video_analyzer.calibrated:
        print("✅ Calibration complete.")
    else:
        print("⚠️ Calibration failed. Using default center. The results may be less accurate.")

    # Start Audio Recording in Parallel
    stop_audio_event = threading.Event()
    audio_thread = threading.Thread(target=record_audio, args=(stop_audio_event, "output_audio.wav"))
    audio_thread.start()

    print("🎥 Interview recording... Press 'q' in the video window to stop.")

    # Main monitoring loop
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        preds = fa.get_landmarks(frame.copy())
        if preds is not None:
            warning = video_analyzer.process_landmarks(frame, preds[0])
            # Display real-time gaze warning
            if warning:
                cv2.putText(frame, warning, (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Interview Analysis", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_audio_event.set()
            break

    # Cleanup and Analysis
    cap.release()
    cv2.destroyAllWindows()
    audio_thread.join()
    print("Video and Audio capture complete. Analyzing results...")

    # Get video report and extract score for fusion
    video_report = video_analyzer.final_report()
    video_score = video_report["AI-Use Probability"]

    # Run speech and text analysis
    speech_analyzer = SpeechAnalyzer()
    speech_score, transcript = speech_analyzer.analyze_audio("output_audio.wav")
    text_analyzer = TextAnalyzer()
    text_score = text_analyzer.analyze(transcript)

    # Fuse results
    final_report = fusion_layer(text_score, speech_score, video_score)
    
    # Print detailed reports
    print("\n" + "="*25)
    print("📹 VIDEO ANALYSIS REPORT 📹")
    print("="*25)
    for k, v in video_report.items():
        print(f"{k}: {v}")

    print("\n" + "="*25)
    print("🔥 FINAL FUSION REPORT 🔥")
    print("="*25)
    for k, v in final_report.items():
        print(f"{k.replace('_', ' ').title()}: {v}")
    print("="*25)


if __name__ == "__main__":
    main()



Calibrating gaze... look at the CENTER for 3 seconds.
✅ Calibration complete.
🎥 Interview recording... Press 'q' in the video window to stop.
🎤 Audio recording started...
🎤 Audio recording stopped. Saving file...
🎤 Audio saved to output_audio.wav
Video and Audio capture complete. Analyzing results...
📝 Transcript: romantic scenes hello good morning we will be talking about so today


Device set to use cuda:0



📹 VIDEO ANALYSIS REPORT 📹
Total Time (s): 23.4
Total Blinks: 3
Suspected Head Movements: 3
AI-Use Probability: 0.08

🔥 FINAL FUSION REPORT 🔥
Text Score: 1.0
Speech Score: 0.3
Video Score: 0.08
Overall Score: 0.54
Verdict: Uncertain


In [17]:
import cv2
import face_alignment
import numpy as np
import threading
import wave
import pyaudio
import time
import librosa
import speech_recognition as sr
from scipy.spatial import distance
from collections import deque
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# ==================================
# Video Analyzer (UPDATED based on your new code)
# ==================================
class VideoAnalyzer:
    def __init__(self):
        # General
        self.start_time = time.time()

        # Blink Detection
        self.total_blinks = 0
        self.prev_ear = 0
        self.blink_thresh = 0.2275

        # Head Movement
        self.prev_head_dir = 0
        self.head_turn_thresh = 0.25
        self.sus_head_movements = 0

        # Gaze Tracking
        self.gaze_history = deque(maxlen=10)
        self.gaze_counts = {"LEFT": 0, "RIGHT": 0, "CENTER": 0}
        self.gaze_warnings = 0

        # Calibration
        self.center_x = 0.5  # default normalized center
        self.calibrated = False

    def eye_aspect_ratio(self, eye):
        """Compute EAR for blink detection."""
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        return (A + B) / (2.0 * C)

    def crop_eye(self, frame, eye_points):
        """Crop the eye region using bounding rect."""
        x_min = int(np.min(eye_points[:, 0]))
        x_max = int(np.max(eye_points[:, 0]))
        y_min = int(np.min(eye_points[:, 1]))
        y_max = int(np.max(eye_points[:, 1]))
        return frame[y_min:y_max, x_min:x_max]

    def detect_pupil(self, eye_img):
        """Detect pupil center using threshold + contours."""
        if eye_img.size == 0:
            return None
        gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            return None

        # Largest contour = pupil
        c = max(contours, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
        return (x + w // 2, y + h // 2)

    def get_head_direction(self, landmarks):
        """Rough head roll detection (nose-chin slope)."""
        nose, chin = landmarks[30], landmarks[8]
        return np.arctan2(chin[1] - nose[1], chin[0] - nose[0])

    def calibrate_center(self, frame, landmarks):
        """Calibrate gaze by asking user to look at the center."""
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]
        x_vals = []
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil is not None and eye_img.shape[1] > 0:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                x_vals.append(norm_x)
        if x_vals:
            self.center_x = np.mean(x_vals)
            self.calibrated = True

    def process_landmarks(self, frame, landmarks):
        """Process landmarks to detect blinks, head movement, and eye gaze."""
        left_eye, right_eye = landmarks[36:42], landmarks[42:48]

        # Blink detection
        ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
        if ear < self.blink_thresh and self.prev_ear >= self.blink_thresh:
            self.total_blinks += 1
        self.prev_ear = ear

        # Head movement
        head_dir = self.get_head_direction(landmarks)
        if self.prev_head_dir != 0 and abs(head_dir - self.prev_head_dir) > self.head_turn_thresh:
            self.sus_head_movements += 1
        self.prev_head_dir = head_dir

        # Eye gaze detection
        gaze_direction = "CENTER"
        for eye_points in [left_eye, right_eye]:
            eye_img = self.crop_eye(frame, eye_points)
            pupil = self.detect_pupil(eye_img)
            if pupil is not None and eye_img.shape[1] > 0:
                cx, _ = pupil
                norm_x = cx / (eye_img.shape[1] + 1e-6)
                self.gaze_history.append(norm_x)

        if len(self.gaze_history) > 0:
            avg_x = np.mean(self.gaze_history)
            if avg_x < self.center_x - 0.1:
                gaze_direction = "RIGHT"  # User looks right, pupils move to left of eye frame
            elif avg_x > self.center_x + 0.06:
                gaze_direction = "LEFT" # User looks left, pupils move to right of eye frame
            else:
                gaze_direction = "CENTER"
            self.gaze_counts[gaze_direction] += 1

        # Return a warning for immediate feedback, only for off-center gaze
        if gaze_direction in ["LEFT", "RIGHT"]:
            self.gaze_warnings += 1
            return f"⚠ GAZE WARNING: {gaze_direction}"
        return None

    def final_report(self):
        """Generate a final dictionary with all video analysis stats."""
        elapsed = round(time.time() - self.start_time, 2)
        # Simple AI-use probability scoring
        prob_ai = (
            (self.gaze_warnings * 0.4) +
            (self.sus_head_movements * 0.3) +
            (self.total_blinks * 0.3)
        )
        prob_ai = min(prob_ai / 100.0, 1.0)  # Normalize 0–1

        return {
            "Total Time (s)": elapsed,
            "Total Blinks": self.total_blinks,
            "Suspected Head Movements": self.sus_head_movements,
            "AI-Use Probability": round(prob_ai, 2)
        }

# ==================================
# Audio Recorder (No changes needed)
# ==================================
def record_audio(stop_event, filename="output_audio.wav"):
    FORMAT, CHANNELS, RATE, CHUNK = pyaudio.paInt16, 1, 16000, 1024
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("🎤 Audio recording started...")
    frames = []
    while not stop_event.is_set():
        data = stream.read(CHUNK)
        frames.append(data)
    print("🎤 Audio recording stopped. Saving file...")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    print(f"🎤 Audio saved to {filename}")


# ==================================
# Speech Analyzer (No changes needed)
# ==================================
class SpeechAnalyzer:
    def __init__(self):
        self.score = 0.0

    def analyze_audio(self, filename="output_audio.wav"):
        try:
            r = sr.Recognizer()
            with sr.AudioFile(filename) as source:
                audio = r.record(source)
            text = r.recognize_google(audio)
            print("📝 Transcript:", text)
            y, sr_rate = librosa.load(filename, sr=None)
            pitch = librosa.yin(y, fmin=50, fmax=300)
            pitch_std = np.std(pitch[~np.isnan(pitch)])
            if pitch_std < 5: self.score += 0.4
            if len(y) > 0 and len(text.split()) / (len(y) / sr_rate) < 1.5: self.score += 0.3
            if not text: self.score += 0.3
            return round(min(self.score, 1.0), 2), text
        except Exception as e:
            print(f"Speech analysis failed: {e}. Returning default values.")
            return 0.5, ""


# ==================================
# Text Analyzer (UPDATED with your new code)
# ==================================
class TextAnalyzer:
    def __init__(self, model_path="./final_model"):
        """Loads the fine-tuned model and creates a text-classification pipeline."""
        self.detector = None
        try:
            print(f"Loading model from: {model_path}")
            tokenizer = AutoTokenizer.from_pretrained(model_path)
            model = AutoModelForSequenceClassification.from_pretrained(model_path)
            self.detector = pipeline("text-classification", model=model, tokenizer=tokenizer)
            print("🤖 Text analysis pipeline created successfully with your fine-tuned model.")
        except OSError:
            print(f"ERROR: Model not found at '{model_path}'.")
            print("Text analysis will be skipped. Please check the model path.")


    def analyze(self, text):
        """Analyzes the text using the fine-tuned model."""
        # If the model failed to load or there's no text, return a neutral score.
        if self.detector is None or not text.strip():
            return 0.5
        
        # Use the pipeline to get the prediction
        result = self.detector(text)
        prediction = result[0]
        label = prediction['label']
        score = prediction['score']

        print(f"Text Analysis Prediction: Label={label}, Confidence={score:.2%}")

        # Convert the label and score to a single AI-probability score
        # This assumes your model's labels are like 'AI-Generated' and 'Human-Written'
        # Adjust the label name if your model uses different ones (e.g., LABEL_1, LABEL_0)
        if "AI" in label.upper() or "MACHINE" in label.upper():
            return round(score, 2)
        else: # Assumes the other label is Human-Written
            return round(1 - score, 2)


# ==================================
# Fusion Layer (No changes needed)
# ==================================
def fusion_layer(text_score, speech_score, video_score):
    overall_score = 0.4 * text_score + 0.4 * speech_score + 0.2 * video_score
    label = "Uncertain"
    if overall_score < 0.45:
        label = "Likely Human"
    elif overall_score > 0.65:
        label = "Likely AI / Suspicious"
    return {
        "text_score": text_score,
        "speech_score": speech_score,
        "video_score": video_score,
        "overall_score": round(overall_score, 2),
        "verdict": label
    }


# ==================================
# Main Pipeline (UPDATED)
# ==================================
def main():
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu')
    video_analyzer = VideoAnalyzer()
    cap = cv2.VideoCapture(0)

    # Calibration phase
    print("Calibrating gaze... look at the CENTER for 3 seconds.")
    start_calib = time.time()
    while time.time() - start_calib < 3:
        ret, frame = cap.read()
        if not ret: break
        preds = fa.get_landmarks(frame)
        if preds is not None:
            video_analyzer.calibrate_center(frame, preds[0])
        cv2.putText(frame, "Look at CENTER for calibration", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    
    cv2.destroyWindow("Calibration")
    if video_analyzer.calibrated:
        print("✅ Calibration complete.")
    else:
        print("⚠️ Calibration failed. Using default center. The results may be less accurate.")

    # Start Audio Recording in Parallel
    stop_audio_event = threading.Event()
    audio_thread = threading.Thread(target=record_audio, args=(stop_audio_event, "output_audio.wav"))
    audio_thread.start()

    print("🎥 Interview recording... Press 'q' in the video window to stop.")

    # Main monitoring loop
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        preds = fa.get_landmarks(frame.copy())
        if preds is not None:
            warning = video_analyzer.process_landmarks(frame, preds[0])
            # Display real-time gaze warning
            if warning:
                cv2.putText(frame, warning, (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Interview Analysis", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_audio_event.set()
            break

    # Cleanup and Analysis
    cap.release()
    cv2.destroyAllWindows()
    audio_thread.join()
    print("Video and Audio capture complete. Analyzing results...")

    # Get video report and extract score for fusion
    video_report = video_analyzer.final_report()
    video_score = video_report["AI-Use Probability"]

    # Run speech and text analysis
    speech_analyzer = SpeechAnalyzer()
    speech_score, transcript = speech_analyzer.analyze_audio("output_audio.wav")
    text_analyzer = TextAnalyzer() # Will load your fine-tuned model
    text_score = text_analyzer.analyze(transcript)

    # Fuse results
    final_report = fusion_layer(text_score, speech_score, video_score)
    
    # Print detailed reports
    print("\n" + "="*25)
    print("📹 VIDEO ANALYSIS REPORT 📹")
    print("="*25)
    for k, v in video_report.items():
        print(f"{k}: {v}")

    print("\n" + "="*25)
    print("🔥 FINAL FUSION REPORT 🔥")
    print("="*25)
    for k, v in final_report.items():
        print(f"{k.replace('_', ' ').title()}: {v}")
    print("="*25)


if __name__ == "__main__":
    main()



Calibrating gaze... look at the CENTER for 3 seconds.
✅ Calibration complete.
🎥 Interview recording... Press 'q' in the video window to stop.
🎤 Audio recording started...
🎤 Audio recording stopped. Saving file...
🎤 Audio saved to output_audio.wav


Device set to use cuda:0


Video and Audio capture complete. Analyzing results...
Speech analysis failed: recognition connection failed: [Errno 11001] getaddrinfo failed. Returning default values.
Loading model from: ./final_model
🤖 Text analysis pipeline created successfully with your fine-tuned model.

📹 VIDEO ANALYSIS REPORT 📹
Total Time (s): 28.73
Total Blinks: 7
Suspected Head Movements: 0
AI-Use Probability: 0.03

🔥 FINAL FUSION REPORT 🔥
Text Score: 0.5
Speech Score: 0.5
Video Score: 0.03
Overall Score: 0.41
Verdict: Likely Human
